In [1]:
%load_ext lab_black

In [2]:
import altair as alt
import altair_stiles as altstiles

alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [3]:
import pandas as pd
import us

In [4]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread

In [5]:
from df2gspread import df2gspread as d2g

### datawrapper credentials

In [6]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

### setting up google sheets

In [7]:
# scope = ['https://spreadsheets.google.com/feeds']
# credentials = ServiceAccountCredentials.from_json_keyfile_name('jupyter-integration-349314-25735d35924b.json', scope)
# gc = gspread.authorize(credentials)

### get a specific spreadsheet

In [8]:
# spreadsheet_key = "1LXH-tda6l5Mx53bJHUDyDJrAZxifp_0O7F3P47BgtIo"
# book = gc.open_by_key(spreadsheet_key)

In [9]:
# Opening the worksheet by using Worksheet ID
# workbook = gc.open_by_key(spreadsheet_key)
# Selecting which sheet to pulling the data
# sheet = workbook.worksheet('Sheet1')
# Pulling the data and transform it to the data frame
# values = sheet.get_all_values()
# pacifictemps = pd.DataFrame(values[1:], columns = values[0])

In [10]:
import requests
import bs4

In [11]:
raw_military_aid = pd.read_csv("data/raw/aid_countries_final.csv")

In [12]:
raw_military_aid.columns = raw_military_aid.columns.str.lower().str.replace(" ", "_")

In [13]:
raw_military_aid.head()

,country_id,country_code,country_name,region_id,region_name,income_group_id,income_group_name,income_group_acronym,managing_agency_id,managing_agency_acronym,...,activity_name,activity_description,activity_project_number,activity_start_date,activity_end_date,transaction_type_id,transaction_type_name,fiscal_year,current_dollar_amount,constant_dollar_amount
0,4,AFG,Afghanistan,4,South and Central Asia,1,Low Income Country,LIC,7,DOD,...,The FA.gov team redacted this field with the e...,The FA.gov team redacted this field with the e...,NaN,NaN,NaN,2,Obligations,2022,1218364000,1176367674
1,4,AFG,Afghanistan,4,South and Central Asia,1,Low Income Country,LIC,7,DOD,...,The FA.gov team redacted this field with the e...,The FA.gov team redacted this field with the e...,NaN,NaN,NaN,2,Obligations,2022,237119000,228945641
2,400,JOR,Jordan,3,Middle East and North Africa,3,Upper Middle Income Country,UMIC,1,USAID,...,Direct economic support to the Government of J...,The Syria crisis and the resulting influx of r...,278-CDCS-JORDAN-CT-FY21-00-03,NaN,NaN,2,Obligations,2022,180100000,173892054
3,400,JOR,Jordan,3,Middle East and North Africa,3,Upper Middle Income Country,UMIC,1,USAID,...,Emergency Food Assistance,Food assistance to refugees.,720BHA22IO00004,NaN,2022-10-13,2,Obligations,2022,52000000,50207589
4,400,JOR,Jordan,3,Middle East and North Africa,3,Upper Middle Income Country,UMIC,1,USAID,...,Direct economic support to the Government of J...,The Syria crisis and the resulting influx of r...,278-CDCS-JORDAN-CT-FY21-00-03,NaN,NaN,2,Obligations,2022,50000000,48276528


In [14]:
byyear = (
    raw_military_aid.groupby(
        ["country_id", "country_code", "country_name", "fiscal_year"]
    )
    .agg(annual_aid=pd.NamedAgg(column="current_dollar_amount", aggfunc=sum))
    .reset_index()
)

In [15]:
wide_format = byyear.pivot(
    index="country_name",
    columns="fiscal_year",
    values="annual_aid",
).reset_index()

In [16]:
dw.add_data(chart_id="shDBL", data=wide_format)

<Response [204]>

In [17]:
raw_military_aid[
    (raw_military_aid["country_name"] == "Ukraine")
    & (raw_military_aid["fiscal_year"] == 2022)
].value_counts("managing_agency_acronym")

managing_agency_acronym
USAID    48
HHS      33
TREAS    11
PC        5
dtype: int64

In [18]:
complete_aid = pd.read_csv("data/raw/us_foreign_aid_complete.csv")

/var/folders/gg/6wnjn6cd1qlfst8mkw4g4xqc0000gn/T/ipykernel_46155/4007346929.py:1: DtypeWarning: Columns (6,7,12,46,51) have mixed types. Specify dtype option on import or set low_memory=False.
  complete_aid = pd.read_csv("data/raw/us_foreign_aid_complete.csv")


In [19]:
complete_aid.columns = complete_aid.columns.str.lower().str.replace(" ", "_")

In [20]:
ukraine_2022 = complete_aid[
    (complete_aid["country_name"] == "Ukraine") & (complete_aid["fiscal_year"] == 2022)
]

In [21]:
dates = ["01MAR2022", "01FEB2022"]

In [22]:
ukraine_2022[ukraine_2022["transaction_date"].isin(dates)].groupby("country_name").agg(
    total_funding=pd.NamedAgg(column="current_dollar_amount", aggfunc=sum)
)

,total_funding
country_name,
Ukraine,163852039


In [23]:
military_aid_2020 = pd.read_csv("data/raw/mil_aid_2020_wegypt.csv")

In [24]:
military_aid_2020.columns = military_aid_2020.columns.str.lower().str.replace(" ", "_")

In [25]:
dw_2020_mil_aid = (
    military_aid_2020.groupby("country_name")
    .agg(mil_aid_20=pd.NamedAgg(column="current_dollar_amount", aggfunc=sum))
    .reset_index()
)

In [26]:
dw.add_data(chart_id="P3SOT", data=dw_2020_mil_aid)

<Response [204]>